## Join the Duet Server the Data Owner 1 connected to

In [1]:
import syft as sy

In [2]:
duet1 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


In [2]:
duet1 = sy.join_duet("f246064f24d360c68e7b410770dcc17d")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: 7b065fe7f5cbdd5e7a3a0221c36b5b35

♫♫♫ > ...waiting for partner to connect...

♫♫♫ > CONNECTED!


## Join the Duet Server the Data Owner 2 connected to

In [3]:
duet2 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


## Get data from Duet Server

In [4]:
duet1.store.pandas

,ID,Tags,Description,object_type
0,<UID: 525378a8d9624a9e886c8b2f3068baac>,[X_Train_Owner1],"Dataset of 400 samples, 7 feature",<class 'torch.Tensor'>
1,<UID: 854a07c3bf394e44b5625c16364f9755>,[y_Train_Owner1],"Dataset of 400 samples, 1 feature",<class 'torch.Tensor'>
2,<UID: f265f31823664e1788e931aadd2b2dbc>,[X_Test_Owner1],"Dataset of 400 samples, 7 feature",<class 'torch.Tensor'>
3,<UID: 58deaf7d9f6140f084971563bd5da3e2>,[y_Test_Owner1],"Dataset of 400 samples, 1 feature",<class 'torch.Tensor'>


In [5]:
X_Train_Owner1_ptr = duet1.store[0]
y_Train_Owner1_ptr = duet1.store[1]
X_Test_Owner1_ptr = duet1.store[2]
y_Test_Owner1_ptr = duet1.store[3]

print(X_Train_Owner1_ptr.tags)
print(X_Train_Owner1_ptr.description)

['X_Train_Owner1']
Dataset of 400 samples, 7 feature


In [6]:
duet2.store.pandas

,ID,Tags,Description,object_type
0,<UID: 6dc8858a926a4745924b767ce88e7273>,[X_Train_Owner2],"Dataset of 400 samples, 7 feature",<class 'torch.Tensor'>
1,<UID: 4184124411db439bb47960409c2bb9ef>,[y_Train_Owner2],"Dataset of 400 samples, 1 feature",<class 'torch.Tensor'>
2,<UID: 78f059281d694172973cb23a78e5168e>,[X_Test_Owner2],"Dataset of 400 samples, 7 feature",<class 'torch.Tensor'>
3,<UID: 21b40ddd1a614b58bc5ad245f7d20005>,[y_Test_Owner2],"Dataset of 400 samples, 1 feature",<class 'torch.Tensor'>


In [7]:
X_Train_Owner2_ptr = duet2.store[0]
y_Train_Owner2_ptr = duet2.store[1]
X_Test_Owner2_ptr = duet2.store[2]
y_Test_Owner2_ptr = duet2.store[3]

print(X_Train_Owner2_ptr.tags)
print(X_Train_Owner2_ptr.description)

['X_Train_Owner2']
Dataset of 400 samples, 7 feature


## Build a Base Classifier Model

In [8]:
class SyNet(sy.Module):
    def __init__(self,input_channels,output_channels,torch_ref):
        super(SyNet,self).__init__(torch_ref=torch_ref)
        #Our network:
        # Linear1->relu->Batchnorm->Linear2->relu->Batchnorm->Dropout->Linear3->output
        # Softmax is added in the predict function
        #This applies Linear transformation to input data. 
        self.fc1 = self.torch_ref.nn.Linear(input_channels,int(1.5*input_channels))
        self.fc2 = self.torch_ref.nn.Linear(int(1.5*input_channels),int(1.5*input_channels))
        self.fc3 = self.torch_ref.nn.Linear(int(1.5*input_channels),output_channels)
        
        self.relu = self.torch_ref.nn.ReLU()
        self.dropout = self.torch_ref.nn.Dropout(p=0.1)
        self.batchnorm1 = self.torch_ref.nn.BatchNorm1d(int(1.5*input_channels))
        self.batchnorm2 = self.torch_ref.nn.BatchNorm1d(int(1.5*input_channels))
        self.sigmoid = self.torch_ref.nn.Sigmoid()
    #This must be implemented
    def forward(self,x):
        x = self.relu(self.fc1(x))
        x = self.batchnorm1(x)
        x = self.relu(self.fc2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x
    def predict(self,x):
        output = self.forward(x)
        prediction = self.torch_ref.argmax(output,1)
        return prediction

In [9]:
def classifier_train(epochs, model, optimizer, X_ptr,y_ptr,criterion,torch_ref):
    losses = []
    for i in range(epochs):
        #Precit the output for Given input
        y_pred_ptr = model.forward(X_ptr)
        #Compute Cross entropy loss
        loss = criterion(y_pred_ptr,y_ptr)
        loss_item = loss.item()
        #Request to get the loss value
        loss_value = loss_item.get(
            reason="To evaluate training progress",
            request_block=True,
            timeout_secs=5,
        )
        #Add loss to the list
        losses.append(loss_value)
        #Print loss
        if i%50==0:
            print("Epoch:",i," Loss:",loss_value)
        
        #Clear the previous gradients
        optimizer.zero_grad()
        #Compute gradients
        loss.backward()
        #Adjust weights
        optimizer.step()
    return losses

In [10]:
import torch as base_torch         
import numpy as np

In [11]:
# Base Classifier Model
X_Train_Owner1_shape = X_Train_Owner1_ptr.shape.get(
            reason="To evaluate training progress",
            request_block=True,
            timeout_secs=5,
        )
input_channels = X_Train_Owner1_shape[1]
output_channels = 2
base_model = SyNet(input_channels,output_channels,base_torch)

## Send copy of model to Data Owner1and train them remotely

In [12]:
remote_model1 = base_model.send(duet1)

remote_torch1 = duet1.torch
params = remote_model1.parameters()
#Define loss criterion
criterion = remote_torch1.nn.CrossEntropyLoss()
#Define the optimizer
optimizer = remote_torch1.optim.Adam(params=params, lr=0.001)
#Number of epochs
epochs = 1000

# Training
losses = classifier_train(epochs, remote_model1, optimizer, X_Train_Owner1_ptr, y_Train_Owner1_ptr,criterion, remote_torch1)

Epoch: 0  Loss: 0.7004346251487732
Epoch: 50  Loss: 0.6451839208602905
Epoch: 100  Loss: 0.6058306694030762
Epoch: 150  Loss: 0.5695232152938843
Epoch: 200  Loss: 0.5338520407676697
Epoch: 250  Loss: 0.5037806034088135
Epoch: 300  Loss: 0.48326438665390015
Epoch: 350  Loss: 0.4705788791179657
Epoch: 400  Loss: 0.4617140591144562
Epoch: 450  Loss: 0.44991788268089294
Epoch: 500  Loss: 0.4418407082557678
Epoch: 550  Loss: 0.44389012455940247
Epoch: 600  Loss: 0.4369836449623108
Epoch: 650  Loss: 0.4358045160770416
Epoch: 700  Loss: 0.43021854758262634
Epoch: 750  Loss: 0.4276730716228485
Epoch: 800  Loss: 0.4326799809932709
Epoch: 850  Loss: 0.4234877824783325
Epoch: 900  Loss: 0.42232897877693176
Epoch: 950  Loss: 0.41833141446113586


In [84]:
#y_Test_Owner1_predict_ptr = remote_model1.predict(X_Test_Owner1_ptr)
#s = y_Test_Owner1_predict_ptr-y_Test_Owner1_ptr
#res = s.get(reason="To evaluate training progress",
#             request_block=True,
#             timeout_secs=5,
#         )

## Send copy of model to Data Owner2and train them remotely

In [13]:
remote_model2 = base_model.send(duet2)

remote_torch2 = duet2.torch
params = remote_model2.parameters()
#Define loss criterion
criterion = remote_torch2.nn.CrossEntropyLoss()
#Define the optimizer
optimizer = remote_torch2.optim.Adam(params=params, lr=0.001)
#Number of epochs
epochs = 1000

# Training
losses = classifier_train(epochs, remote_model2, optimizer, X_Train_Owner2_ptr, y_Train_Owner2_ptr,criterion, remote_torch2)

Epoch: 0  Loss: 0.6966527700424194
Epoch: 50  Loss: 0.6478937268257141
Epoch: 100  Loss: 0.609880268573761
Epoch: 150  Loss: 0.577053964138031
Epoch: 200  Loss: 0.5489195585250854
Epoch: 250  Loss: 0.5302548408508301
Epoch: 300  Loss: 0.5106964707374573
Epoch: 350  Loss: 0.5011875033378601
Epoch: 400  Loss: 0.4858357310295105
Epoch: 450  Loss: 0.4717283546924591
Epoch: 500  Loss: 0.47332873940467834
Epoch: 550  Loss: 0.470691978931427
Epoch: 600  Loss: 0.4606500267982483
Epoch: 650  Loss: 0.45962849259376526
Epoch: 700  Loss: 0.45874762535095215
Epoch: 750  Loss: 0.4518145024776459
Epoch: 800  Loss: 0.4484209716320038
Epoch: 850  Loss: 0.4407496750354767
Epoch: 900  Loss: 0.4460342526435852
Epoch: 950  Loss: 0.43799644708633423


## Averaging Model Updates

In [14]:
from collections import OrderedDict

In [15]:
param1 = remote_model1.parameters().get(request_block=True)
param2 = remote_model2.parameters().get(request_block=True)

print("Base Model parameters:")
print(base_model.parameters())
print()

print("Remote model1 parameters:")
print(param1)
print()

print("Remote model2 parameters:")
print(param2)

Base Model parameters:
[Parameter containing:
tensor([[ 0.1314, -0.0100, -0.1370, -0.1415, -0.1813, -0.1167, -0.3393],
        [ 0.0680, -0.2117,  0.1996,  0.1533,  0.2154,  0.1933, -0.0834],
        [-0.2420,  0.0155,  0.1908, -0.2892,  0.0289, -0.2381, -0.1863],
        [-0.2485, -0.1510,  0.2578,  0.1517,  0.1352, -0.0104, -0.0066],
        [-0.1201, -0.0559, -0.1008,  0.2836, -0.1717, -0.2393, -0.2102],
        [-0.1682, -0.0529, -0.2872, -0.0122, -0.1079,  0.0922,  0.0068],
        [-0.2889,  0.0480, -0.2706, -0.2120, -0.0601, -0.3508,  0.2083],
        [-0.1316, -0.3094, -0.3572, -0.0752, -0.3105, -0.2696, -0.1861],
        [-0.3545,  0.3605, -0.0287, -0.3060,  0.3383,  0.0435,  0.2597],
        [ 0.0729, -0.0745,  0.1798, -0.3712, -0.1393,  0.0786,  0.2052]],
       requires_grad=True), Parameter containing:
tensor([-0.0526,  0.0100, -0.2898, -0.1563, -0.0824,  0.3321,  0.1174, -0.3229,
        -0.0069, -0.0471], requires_grad=True), Parameter containing:
tensor([[ 0.0279, -0.09

In [16]:
remote_model1_updates = remote_model1.get(
    request_block=True
).state_dict()

print(remote_model1_updates)

remote_model2_updates = remote_model2.get(
    request_block=True
).state_dict()

print(remote_model2_updates)

OrderedDict([('fc1.weight', tensor([[-9.3534e-03, -7.9372e-02, -2.2492e-01, -2.4150e-01, -2.7234e-01,
         -9.3426e-02, -1.7859e-01],
        [ 2.7532e-01, -6.0084e-02,  2.8861e-01,  6.4675e-02,  1.7945e-01,
          4.7683e-04,  4.5436e-02],
        [-8.1272e-02, -1.6110e-01,  1.3565e-01, -4.6696e-01,  1.5082e-01,
         -2.8951e-01,  1.7403e-02],
        [-1.2627e-02, -2.2554e-01,  2.2431e-01,  1.4033e-01,  6.1436e-02,
          2.0859e-01,  7.6308e-02],
        [-5.1726e-02, -9.9242e-03, -7.5698e-02,  1.2318e-01, -2.2598e-01,
         -3.3482e-01, -2.4864e-01],
        [-1.1376e-01,  3.1022e-02, -1.1888e-01, -3.0595e-02, -3.1798e-01,
          2.2387e-01, -5.1228e-02],
        [-3.2334e-01, -2.5574e-03, -9.0825e-02, -4.5668e-03, -4.2203e-01,
         -2.7367e-01, -7.8237e-02],
        [-3.5973e-01, -1.2180e-02, -3.2763e-01, -2.0225e-01, -4.4690e-01,
         -2.7135e-01, -2.9107e-01],
        [-3.7991e-01,  1.4592e-01,  1.1175e-01, -1.4534e-01,  5.2560e-01,
         -2.6800e-

In [17]:
# Get Average Model
avg_updates = OrderedDict()
for key in remote_model1_updates.keys():
    avg_updates[key] = (remote_model1_updates[key] + remote_model2_updates[key]) / 2

    
combined_model = SyNet(input_channels,output_channels,base_torch)

combined_model.load_state_dict(avg_updates)

## Test the prediction of combined model

In [18]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

german = pd.read_csv("./data/german_data_2.csv")  

In [19]:
test_size = 0.20

processed_data = None
categorical = None
label_encoders = {}

def preprocessing(dataset, data, test_size):
    """
    Preprocess dataset

    Parameters
    ----------
    data: DataFrame
        Pandas dataframe containing German dataset.
    """
    
    global processed_data
    global categorical
    global label_encoders

    # Reset global variables
    
    processed_data = None
    categorical = None
    label_encoders = {}


    if dataset == "German":
        # Drop savings account and checkings account columns as they contain a lot
        # of NaN values and may not always be available in real life scenarios
        data = data.drop(columns = ['Saving accounts', 'Checking account'])
        
    dat_dict = data.to_dict()
    new_dat_dict = {}

    # rename columns(Make them lowercase and snakecase)
    for key, value in dat_dict.items():
        newKey = key
        if type(key) == str:
            newKey = newKey.lower().replace(' ', '_')
        # if newKey != key:
        new_dat_dict[newKey] = dat_dict[key]
    del dat_dict

    data = pd.DataFrame.from_dict(new_dat_dict)
    del new_dat_dict


    # print(data.describe())
    # print(data.describe(include='O'))

    cols = data.columns
    num_cols = data._get_numeric_data().columns
    categorical = list(set(cols) - set(num_cols))

    # Drop null rows
    data = data.dropna()

    # Encode text columns to number values
    for category in categorical:
        le = LabelEncoder()
        data[category] = le.fit_transform(data[category])
        label_encoders[category] = le

    for col in data.columns:
        if(col not in categorical):
            data[col] = (data[col].astype('float') - np.mean(data[col].astype('float')))/np.std(data[col].astype('float'))

    # print(data.describe())
    # print(data.describe(include='O'))

    processed_data = data

    # Get Training parameters
    if dataset == "German":
        target_col = data.columns[-1]
        x = data.drop(columns=target_col, axis=1)
        y = data[target_col].astype('int')
    elif dataset == "Australian":
        x = data.drop(14, axis=1)
        y = data[14].astype('int')
    elif dataset == "Japanese":
        x = data.drop(15, axis=1)
        y = data[15].astype('int')
    elif dataset == "Taiwan":
        x = data.drop('default_payment_next_month', axis=1)
        y = data['default_payment_next_month'].astype('int')
    elif dataset == "Polish":
        x = data.drop('class', axis=1)
        y = data['class'].astype('int')


    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = test_size)
    x_train = pd.DataFrame(x_train)
    y_train = pd.DataFrame(y_train)
    y_test = pd.DataFrame(y_test)

    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)
    
    y_train = y_train[y_train.columns[0]].to_numpy()
    y_test = y_test[y_test.columns[0]].to_numpy()

    return (x_train, x_test, y_train, y_test)

X_train, X_test, y_train, y_test = preprocessing("German", german, test_size)

In [20]:
# Trnasform the input to tensor
X_test_tensor = base_torch.FloatTensor(X_test)
y_test_tensor = base_torch.tensor(y_test,dtype=base_torch.long)

In [21]:
# Calcualte Accuracy
from sklearn.metrics import accuracy_score

def accuracy(model,X,y):
    print(accuracy_score(model.predict(X),y))

In [22]:
accuracy(combined_model,X_test_tensor,y_test_tensor)

0.74
